In [1]:
import pandas as pd
from IPython.display import display

# Load the datasets
ground_truth = pd.read_csv(r"data\dat_dialite\Categorical - Review\groundreview.csv")
alite_output = pd.read_csv(r"data\dat_dialite\Categorical - Review\ir_alite_Categorical - Reviews.csv")

# Step 1: Standardise column names
ground_truth.columns = ground_truth.columns.str.strip().str.lower()
alite_output.columns = alite_output.columns.str.strip().str.lower()
print(ground_truth.columns)
print(alite_output.columns)

# Step 2: Identify common columns
common_cols = sorted(set(ground_truth.columns) & set(alite_output.columns))
print("Common columns:", common_cols)

# Step 3: Clean and flatten each row into a tuple of values
def clean_and_flatten(df):
    return [
        tuple(
            str(x).strip().lower() if pd.notna(x) else "missing"
            for x in row
        )
        for row in df[common_cols].values.tolist()
    ]

gt_rows = set(clean_and_flatten(ground_truth))
ao_rows = set(clean_and_flatten(alite_output))

# Step 4: Compare the sets of rows
tp = gt_rows & ao_rows  # True Positives
fp = ao_rows - gt_rows  # False Positives
fn = gt_rows - ao_rows  # False Negatives

print("True Positives:", len(tp))
print("False Positives:", len(fp))
print("False Negatives:", len(fn))

# Step 5: Compute metrics
precision = len(tp) / (len(tp) + len(fp)) if (len(tp) + len(fp)) > 0 else 0
recall = len(tp) / (len(tp) + len(fn)) if (len(tp) + len(fn)) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print("\nPrecision: {:.2%}".format(precision))
print("Recall:    {:.2%}".format(recall))
print("F1 Score:  {:.2%}".format(f1))

# Step 6: Display mismatches safely
fp_rows = [list(row) for row in fp if len(row) == len(common_cols)]
fn_rows = [list(row) for row in fn if len(row) == len(common_cols)]

fp_df = pd.DataFrame(fp_rows, columns=common_cols)
fn_df = pd.DataFrame(fn_rows, columns=common_cols)

print("\nRows in ALITE but not in Ground Truth (False Positives):")


print("\nRows in Ground Truth but missing from ALITE (False Negatives):")


# Step 7: Save metrics
metrics = pd.DataFrame({
    'Metric': ['Precision', 'Recall', 'F1 Score'],
    'Value': [precision, recall, f1]
})
metrics.to_csv("analysis/set_row_match_metrics.csv", index=False)

Index(['review', 'sentiment', 'title'], dtype='object')
Index(['title', 'review', 'ï»¿review', 'sentiment'], dtype='object')
Common columns: ['review', 'sentiment', 'title']
True Positives: 150
False Positives: 3
False Negatives: 250

Precision: 98.04%
Recall:    37.50%
F1 Score:  54.25%

Rows in ALITE but not in Ground Truth (False Positives):

Rows in Ground Truth but missing from ALITE (False Negatives):


In [19]:
import pandas as pd

# Read your tables
import matplotlib.pyplot as plt

tableA = pd.read_csv(r"data\dat_dialite\NumericalX2 - Weather\pressure.csv")
tableB = pd.read_csv(r"data\dat_dialite\NumericalX2 - Weather\temperature.csv")
integrated_table = pd.read_csv(r"data\dat_dialite\NumericalX2 - Weather\temp_pressure.csv")
ground_truth_table = pd.read_csv(r"data\dat_dialite\NumericalX2 - Weather\ground_temp_pressure.csv")

# Define tables before integration
tables_before_integration = [tableA, tableB]

# Step 1: Calculate total nulls before integration
nulls_before_integration = sum(table.isnull().sum().sum() for table in tables_before_integration)

# Step 2: Calculate total nulls after integration
nulls_after_integration = integrated_table.isnull().sum().sum()

# Step 3: Calculate total nulls in ground truth
ground_truth_nulls = ground_truth_table.isnull().sum().sum()

# Step 4: Calculate Null Value Scores
nvs_integration = nulls_after_integration - nulls_before_integration
nvs_groundtruth = ground_truth_nulls - nulls_before_integration

# Step 5: Output the results
print("--- Null Value Analysis ---")
print(f"Total Nulls Before Integration: {nulls_before_integration}")
print(f"Total Nulls After Integration: {nulls_after_integration}")
print(f"Ground Truth Nulls: {ground_truth_nulls}")
print()
print(f"Null Value Score (Integrated Table): {nvs_integration}")
print(f"Null Value Score (Ground Truth Table): {nvs_groundtruth}")

# Optional: Calculate percentage change in nulls
if nulls_before_integration > 0:
    pct_change_integration = ((nulls_after_integration - nulls_before_integration) / nulls_before_integration) * 100
    pct_change_groundtruth = ((ground_truth_nulls - nulls_before_integration) / nulls_before_integration) * 100

    print()
    print(f"Percentage Change in Nulls (Integrated Table): {pct_change_integration:.2f}%")
    print(f"Percentage Change in Nulls (Ground Truth Table): {pct_change_groundtruth:.2f}%")
else:
    print("No nulls present before integration; percentage change cannot be calculated.")

# Step 6: Save Results to CSV
results = {
    "Metric": [
        "Total Nulls Before Integration",
        "Total Nulls After Integration",
        "Ground Truth Nulls",
        "Null Value Score (Integrated Table)",
        "Null Value Score (Ground Truth Table)"
    ],
    "Value": [
        nulls_before_integration,
        nulls_after_integration,
        ground_truth_nulls,
        nvs_integration,
        nvs_groundtruth
    ]
}

if nulls_before_integration > 0:
    results["Metric"].extend([
        "Percentage Change in Nulls (Integrated Table)",
        "Percentage Change in Nulls (Ground Truth Table)"
    ])
    results["Value"].extend([
        pct_change_integration,
        pct_change_groundtruth
    ])

results_df = pd.DataFrame(results)
results_df.to_csv("null_value_scores.csv", index=False)
print("\nResults have been saved to 'null_value_scores.csv'.")

--- Null Value Analysis ---
Total Nulls Before Integration: 24710
Total Nulls After Integration: 140
Ground Truth Nulls: 140

Null Value Score (Integrated Table): -24570
Null Value Score (Ground Truth Table): -24570

Percentage Change in Nulls (Integrated Table): -99.43%
Percentage Change in Nulls (Ground Truth Table): -99.43%

Results have been saved to 'null_value_scores.csv'.
